In [1]:
import geopandas as geo
import pandas as pd
from shapely.geometry import Point
from shapely import wkt
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.strtree import STRtree
import numpy as np
import time
from geopandas import sjoin
import gc

In [6]:
gdf = pd.read_csv("Micro_raster_Download_AVERAGE.csv")

In [ ]:
#Teilnehmer brauchen ungefähr 4 Mbits/s DOWNLOAD für Video Konferenzen mit guter Qualität

In [ ]:
# Durchschnittliche Haushaltsgröße ~ 2.2 

In [ ]:
#Zoom Website
#Für Videoanrufe mit zwei Teilnehmern:
#Für qualitativ hochwertiges Video: 600 KBit/s (Upload/Download)
#Für 720p HD-Video: 1,2 MBit/s (Upload/Download)
#Für 1080p HD-Video: 3,8 MBit/s bzw. 3,0 MBit/s (Upload/Download)
#Für Videoanrufe mit Gruppen:
   #Für qualitativ hochwertiges Video: 1,0 MBit/s bzw. 600 KBit/s (Upload/Download)<---------------------------------------
#Für 720p HD-Video: 2,6 MBit/s bzw. 1,8 MBit/s (Upload/Download)
   #Für 1080p HD-Video: 3,8 MBit/s bzw. 3,0 MBit/s (Upload/Download)<---------------------------------------
#Für Empfang der Galerieansicht: 2,0 MBit/s (25 Ansichten), 4,0 MBit/s (49 Ansichten)


In [15]:
gdf.columns[40:146]

Index(['M_11', 'W_01', 'W_02', 'W_03', 'W_04', 'W_05', 'W_06', 'W_07', 'W_08',
       'W_09',
       ...
       'platform', 'model', 'network_name', 'ndt_download_kbit',
       'ndt_upload_kbit', 'signal_strength', 'pinned', 'dtm_level',
       'signal_classification', 'Download_Mean'],
      dtype='object', length=106)

In [8]:
gdf["Durchschnittliche_Haushaltsgröße"] = 0

In [39]:
gdf.loc[gdf["HWS11"] !=0 ,"Durchschnittliche_Haushaltsgröße"] = 2.2

In [162]:
#gdf.loc[((gdf["PHH_1Pers"] + gdf["PHH_2Pers"]  + gdf["PHH_3plusPers"]) !=0), 
#        "Durchschnittliche_Haushaltsgröße"] =  gdf["HWS11"] /  ( gdf["PHH_1Pers"] + gdf["PHH_2Pers"]  + gdf["PHH_3plusPers"] ) 
    

In [40]:
gdf["Download_Zoom_Indicator_HH_normalCALL"] = 0

In [41]:
gdf.loc[(gdf["Download_Mean"] < 600*gdf["Durchschnittliche_Haushaltsgröße"]) 
        &(gdf["Durchschnittliche_Haushaltsgröße"]!=0), "Download_Zoom_Indicator_HH_normalCALL"] = 3

In [42]:
gdf.loc[(gdf["Download_Mean"] > 600*gdf["Durchschnittliche_Haushaltsgröße"]) 
        & (gdf["Download_Mean"] < 1200*gdf["Durchschnittliche_Haushaltsgröße"])
        &(gdf["Durchschnittliche_Haushaltsgröße"]!=0), "Download_Zoom_Indicator_HH_normalCALL"] = 2

In [43]:
gdf.loc[(gdf["Download_Mean"] > 1200*gdf["Durchschnittliche_Haushaltsgröße"])
        &(gdf["Durchschnittliche_Haushaltsgröße"]!=0), "Download_Zoom_Indicator_HH_normalCALL"] = 1

In [50]:
[sum(gdf["Download_Zoom_Indicator_HH_gewichtet"] ==1), 
sum(gdf["Download_Zoom_Indicator_HH_gewichtet"] ==2),
sum(gdf["Download_Zoom_Indicator_HH_gewichtet"] ==3)]

[189312, 6818, 10320]

In [49]:
[sum(gdf["Download_Zoom_Indicator_HH4Pers_gewichtet"] ==1), 
sum(gdf["Download_Zoom_Indicator_HH4Pers_gewichtet"] ==2),
sum(gdf["Download_Zoom_Indicator_HH4Pers_gewichtet"] ==3)]

[160016, 17968, 28463]

In [48]:
[sum(gdf["Download_Zoom_Indicator_HH_normalCALL"] ==1), 
sum(gdf["Download_Zoom_Indicator_HH_normalCALL"] ==2),
sum(gdf["Download_Zoom_Indicator_HH_normalCALL"] ==3)]

[204273, 1243, 934]

In [51]:
gdf = gdf.drop(labels = ['HWS10', 'HWS09',
       'HWS06', 'NWS06', 'Geb06', 'WGeb06', 'Whg06', 'Besch01', 'Arbst01',
       'HH01', 'M_ALL', 'W_ALL', 'ALL_01', 'ALL_02', 'ALL_03', 'ALL_04',
       'ALL_05', 'ALL_06', 'ALL_07', 'ALL_08', 'ALL_09', 'ALL_10', 'ALL_11',
       'M_01', 'M_02', 'M_03', 'M_04', 'M_05', 'M_06', 'M_07', 'M_08', 'M_09',
       'M_10', 'M_11', 'W_01', 'W_02', 'W_03', 'W_04', 'W_05', 'W_06', 'W_07',
       'W_08', 'W_09', 'W_10', 'W_11', 'GEB_AUT', 'GEB_EU', 'GEB_EUN',
       'GEB_EUO', 'GEB_EUS', 'GEB_EUW', 'GEB_EXJ', 'GEB_TUR', 'GEB_DEU',
       'GEB_AFN', 'GEB_AFS', 'GEB_ASW', 'GEB_ASZ', 'GEB_ASO', 'GEB_AMN',
       'GEB_AMS', 'GEB_AUS', 'GEB_XXX', 'STA_AUT', 'STA_EU', 'STA_EUN',
       'STA_EUO', 'STA_EUS', 'STA_EUW', 'STA_EXJ', 'STA_TUR', 'STA_DEU',
       'STA_AFN', 'STA_AFS', 'STA_ASW', 'STA_ASZ', 'STA_ASO', 'STA_AMN',
       'STA_AMS', 'STA_AUS', 'STA_XXX', 'HER_AUT', 'HER_EU', 'HER_EUN',
       'HER_EUO', 'HER_EUS', 'HER_EUW', 'HER_EXJ', 'HER_TUR', 'HER_DEU',
       'HER_AFN', 'HER_AFS', 'HER_ASW','HER_ASZ', 'HER_ASO', 'HER_AMN', 'HER_AMS', 'HER_AUS',
       'HER_XXX', 'ledig', 'verheiratet', 'verwitwet', 'geschieden',
       'Bildung_Akademiker', 'Bildung_Matura', 'Bildung_nichtMatura'], axis =1)

In [46]:
gdf = gdf[gdf["Download_Mean"] != 0]

In [52]:
Grenze = pd.read_csv("Nö_test2.csv")

In [54]:
geometry = Grenze["WKT"].apply(wkt.loads)
Grenze["geometry"] = Grenze["WKT"].apply(wkt.loads)
Grenze = geo.GeoDataFrame(Grenze,geometry =geometry , crs="EPSG:31287")

In [58]:
gdf["geometry"] = gdf["WKT"].apply(wkt.loads)
gdf = geo.GeoDataFrame(gdf,geometry =gdf["geometry"] , crs="EPSG:31287")

In [60]:
gdf=gdf.drop(labels=['index_right'], axis=1)

In [61]:
Nö_gdf = geo.sjoin(gdf, Grenze,how="inner",op="intersects")

In [62]:
Nö_gdf.to_csv("Micro_raster_Zoom_Indikatoren_HH_NÖ+WIEN.csv")

In [ ]:
gdf.to_csv("Micro_raster_Zoom_Indikatoren_HH.csv")